In [2]:
from numpy import save, load
from pandas import read_csv
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import keras.layers 
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model
from sklearn import datasets
from numpy import save, load
from pandas import read_csv
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import Conv1D, SpatialDropout1D
from keras.layers import Convolution1D, Dense,Activation
from keras.models import Input, Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,GlobalAveragePooling1D,TimeDistributed, MaxPooling1D
import tensorflow as tf
import numpy as np
import keras.backend as K
import keras.layers
from keras import optimizers
from keras.layers import Activation, Lambda
from keras.layers import Convolution1D, Dense
from keras.models import Input, Model
from typing import List, Tuple 
from statistics import mean, stdev
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model
from sklearn import datasets

In [3]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
x_train = np.load('gdrive/MyDrive/Colab Notebooks/Dataset2/trainX2.npy')
y_train = np.load('gdrive/My Drive/Colab Notebooks/Dataset2/trainy2.npy')
x_test = np.load('gdrive/My Drive/Colab Notebooks/Dataset2/testX2.npy')
y_test = np.load('gdrive/My Drive/Colab Notebooks/Dataset2/testy2.npy')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted at /content/gdrive
(44339, 128, 6) (44339, 20) (4936, 128, 6) (4936, 20)


In [4]:
x_train = x_train.transpose(0,2,1)
x_test = x_test.transpose(0,2,1)

inputs = np.concatenate((x_train, x_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)
#from sklearn.model_selection import train_test_split
#x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.20, random_state=42)

In [5]:
x_train.shape

(44339, 6, 128)

In [6]:
y_train = (np.argmax(y_train, axis=1)).reshape(-1, 1)
y_test =  (np.argmax(y_test, axis=1)).reshape(-1, 1)

In [7]:
y_train.shape

(44339, 1)

In [8]:
embed_dim = x_train.shape[-1]   # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = x_train.shape[-1]  # Hidden layer size in feed forward network inside transformer

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads,
                                             key_dim=embed_dim)
        self.ffn = keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)  # self-attention layer
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)  # layer norm
        ffn_output = self.ffn(out1)  #feed-forward layer
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)  # layer norm


In [9]:
acc_per_fold = []
loss_per_fold = []
num_folds = 2
inputs = np.concatenate((x_train, x_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)
skf = StratifiedKFold(n_splits=5, shuffle=True)
fold_no = 1
input_shape = x_train.shape[1:]
input = Input(shape=x_train.shape[1:])
for train, test in skf.split(inputs, targets):
    model = keras.Sequential()
    model.add(layers.Input(shape= x_train.shape[1:]))
    model.add(TransformerBlock(embed_dim, num_heads, ff_dim))
    model.add(layers.GlobalAveragePooling1D())
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(ff_dim, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(118, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(inputs[train], targets[train], epochs=200, verbose=True, batch_size=128,callbacks=[keras.callbacks.EarlyStopping(monitor='loss', patience=5,restore_best_weights=True)])
    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

  # Increase fold number
    fold_no = fold_no + 1
model.summary()
callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

Epoch 1/200
308/308 [==============================] - 9s 14ms/step - loss: 1.0994 - accuracy: 0.6916
Epoch 2/200
308/308 [==============================] - 4s 14ms/step - loss: 0.3809 - accuracy: 0.8977
Epoch 3/200
308/308 [==============================] - 4s 13ms/step - loss: 0.2914 - accuracy: 0.9206
Epoch 4/200
308/308 [==============================] - 4s 13ms/step - loss: 0.2486 - accuracy: 0.9313
Epoch 5/200
308/308 [==============================] - 4s 13ms/step - loss: 0.2246 - accuracy: 0.9383
Epoch 6/200
308/308 [==============================] - 4s 13ms/step - loss: 0.2003 - accuracy: 0.9445
Epoch 7/200
308/308 [==============================] - 4s 13ms/step - loss: 0.1889 - accuracy: 0.9478
Epoch 8/200
308/308 [==============================] - 4s 13ms/step - loss: 0.1700 - accuracy: 0.9515
Epoch 9/200
308/308 [==============================] - 4s 13ms/step - loss: 0.1625 - accuracy: 0.9538
Epoch 10/200
308/308 [==============================] - 4s 13ms/step - loss: 0.155

In [10]:
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.1927328109741211 - Accuracy: 96.39776945114136%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.16306741535663605 - Accuracy: 96.32673859596252%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.1699814349412918 - Accuracy: 95.88026404380798%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.19298404455184937 - Accuracy: 96.1440920829773%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.2182117998600006 - Accuracy: 95.78893780708313%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 96.10756039619446 (+- 0.2394906800132919)
> Loss: 0.18739550113677977
--------------------------